In [1]:
import qiskit

/home/hm-tlacherm/.local/lib/python3.6/site-packages/qiskit/__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  DeprecationWarning,


In [2]:
# qiskit.IBMQ.save_account('71bae2b2152c7e73a48f3ebac8d0f76e3274d0b1bcb9e7172e712b67fddf24001c62b2a9df59f42e09da08b46e0d5b81461f73202a5e24121fefe9058d93e5c6')
provider = qiskit.IBMQ.load_account()
#provider.backends()

In [3]:
# useful additional packages
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
from collections import OrderedDict

from qiskit import Aer, IBMQ
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit.optimization.applications.ising import max_cut, tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver, QAOA
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram

from qiskit.algorithms.optimizers import COBYLA, SPSA
from qiskit.opflow import X, Y, Z, I, CX, T, H, S, PrimitiveOp, StateFn
from qiskit.providers.aer import noise

# Import error mitigation functions
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

/home/hm-tlacherm/.local/lib/python3.6/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/hm-tlacherm/.local/lib/python3.6/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


In [4]:
import networkx as nx
import numpy as np
import pylab

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer,  CplexOptimizer, CobylaOptimizer, ADMMOptimizer

from qiskit import BasicAer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import SPSA

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator

from qiskit_optimization.algorithms import WarmStartQAOAOptimizer

# Generate a graph of 4 nodes
n = 5
graph = nx.Graph()
graph.add_nodes_from(np.arange(0, n, 1))
elist = [(0, 3, 9), (0, 4, 6), (1, 2, 9), (1, 4, 10), (2, 4, 7), (3, 4, 7)]
graph.add_weighted_edges_from(elist)

# Compute the weight matrix from the graph
w = nx.adjacency_matrix(graph)

# Formulate the problem as quadratic program
problem = QuadraticProgram()
_ = [problem.binary_var('x{}'.format(i)) for i in range(n)]  # create n binary variables
linear = w.dot(np.ones(n))
quadratic = -w
problem.maximize(linear=linear, quadratic=quadratic)

# Fix node 0 to be 1 to break the symmetry of the max-cut solution
problem.linear_constraint([1, 0, 0, 0, 0], '==', 1)

# Run quantum algorithm QAOA on qasm simulator
spsa = SPSA(maxiter=500)
backend = BasicAer.get_backend('qasm_simulator')

def store_intermediate_result1(eval_count, parameters, mean, std):
    counts1.append(eval_count)
    values1.append(mean)

# Noise
device_backend = 'ibmq_lima'
backend = Aer.get_backend('aer_simulator')
counts1 = []
values1 = []
backend = provider.get_backend('ibmq_lima')
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

qi_noise_model_ibmq = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), 
                    shots=8000,
                    noise_model=noise_model,
                    coupling_map=coupling_map,
                    basis_gates=basis_gates,
)
                #  measurement_error_mitigation_cls=CompleteMeasFitter,
                #  measurement_error_mitigation_shots=8000)

qaoa = QAOA(optimizer=spsa, quantum_instance=qi_noise_model_ibmq, reps=10, callback=store_intermediate_result1, initial_point=[0., 0., 0., 0., 0.,0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,1., 1., 1., 1., 1.])

algorithm = MinimumEigenOptimizer(qaoa)
result = algorithm.solve(problem)
c = qaoa.get_optimal_circuit()
# print(result.samples)
print(result)  # prints solution, x=[1, 0, 1, 0], the cost, fval=4

optimal function value: 35.0
optimal value: [1. 0. 1. 0. 1.]
status: SUCCESS


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:68: DeprecationWarning: 
The VQE.get_optimal_circuit method is deprecated as of Qiskit Terra
0.18.0 and will be removed no sooner than 3 months after the releasedate.
This information is part of the returned result object and can be
queried as VQEResult.ansatz.bind_parameters(VQEResult.optimal_point).


In [5]:
import qiskit.tools.jupyter
%qiskit_version_table